In [1]:
import chess
import chess.engine
import random
import pandas as pd

In [2]:
def evaluate_all_engines(board, color, depths, times, engine):
    results = {}
    for depth in depths:
        for time_limit in times:
            result = engine.analyse(board, chess.engine.Limit(depth=depth, time=time_limit))
            score = result['score'].pov(color=color).score(mate_score=900)
            results[f"{depth}_{time_limit}"] = score
    return results

In [3]:
NUM_GAMES = 200
MOVE_LIMIT = 80

stockfish_path = "D:\PikeBot\stockfish\stockfish-windows-x86-64-avx2.exe"
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
depths = [2, 5, 10, 15, 20]
time_limits = [0.1, 0.01, 0.001, 1e-4, 1e-5]
columns = []
for depth in depths:
    for limit in time_limits:
        columns.append(f"{depth}_{limit}")

results_df = pd.DataFrame(columns=columns)
for i in range(NUM_GAMES):
    board = chess.Board()
    while not board.is_game_over(claim_draw=True):
        legal_moves = list(board.legal_moves)
        random_move = random.choice(legal_moves)
        board.push(random_move)
        result = evaluate_all_engines(board, board.turn, depths, time_limits, engine)
        new_df = pd.DataFrame(result, index=[0])
        results_df = pd.concat([results_df, new_df], ignore_index=True)
        if board.fullmove_number > MOVE_LIMIT:
            break

results_df.head()

,2_0.1,2_0.01,2_0.001,2_0.0001,2_1e-05,5_0.1,5_0.01,5_0.001,5_0.0001,5_1e-05,...,15_0.1,15_0.01,15_0.001,15_0.0001,15_1e-05,20_0.1,20_0.01,20_0.001,20_0.0001,20_1e-05
0,0,32,32,35,35,12,12,12,12,12,...,10,11,11,10,9,5,7,6,6,6
1,79,110,110,110,110,132,202,202,202,202,...,131,128,130,130,130,138,139,144,134,131
2,-35,-24,-24,-24,-24,-47,-44,-44,-44,-44,...,-50,-46,-43,-43,-42,-52,-56,-53,-53,-50
3,91,94,94,94,94,94,94,94,94,94,...,118,120,128,121,121,120,126,128,128,127
4,30,30,30,30,30,43,43,29,29,29,...,30,28,28,21,26,32,32,25,28,21


In [4]:
reference_column = '20_0.1'
difference_results = {}
for col in results_df.columns:
    if col != reference_column:
        absolute_diff = results_df[col].sub(results_df[reference_column]).abs()
        avg_abs_diff = absolute_diff.mean()
        std_abs_diff = absolute_diff.std()
        relative_avg_abs_diff = avg_abs_diff / results_df[reference_column].mean() 
        difference_results[col] = {'Avg_Abs_Diff': avg_abs_diff, 'Std_Abs_Diff': std_abs_diff, 'Relative_Avg_Abs_Diff': relative_avg_abs_diff}

differences_df = pd.DataFrame(difference_results).T

differences_df = differences_df.sort_values(by='Relative_Avg_Abs_Diff')
differences_df

,Avg_Abs_Diff,Std_Abs_Diff,Relative_Avg_Abs_Diff
20_0.01,10.099460,39.731630,0.054677
20_0.001,13.301794,82.565021,0.072014
20_1e-05,13.959208,79.100030,0.075573
20_0.0001,14.391081,100.306057,0.077911
15_0.01,26.060001,50.398928,0.141085
15_1e-05,26.456088,72.326521,0.143230
15_0.0001,26.504518,60.239914,0.143492
15_0.001,27.010726,95.022206,0.146232
15_0.1,29.469609,52.704987,0.159544
10_1e-05,52.269713,72.176501,0.282981
